#### El cliente ha pedido realizar un ajuste a las reglas implementadas hasta ahora, estos ajustes consisten en reemplazar algunas funciones hechas con groupBy por funciones Window, presta mucha atención y resuelve las siguientes actividades.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de las sesiones 06, 07

In [1]:
%%HTML <style>pre { white-space: pre !important; }</style>

In [2]:
// NO MODIFICAR CONTENIDO DE ESTA CELDA
import org.apache.spark.sql.{SparkSession, DataFrame, Column, Row}
import org.apache.spark.sql.{functions => f}
import org.apache.spark.sql.{types => t}
import org.apache.spark.sql.expressions.Window

def difference(l1: Seq[String], l2: Seq[String]): Seq[Column] =
    l1.diff(l2).map(colName => f.col(colName))

def readTmpDf(dfSeq: Seq[String]): Map[String, DataFrame] =
    dfSeq.map(table_name => (table_name, spark.read.parquet("../../resources/data/tmp/parquet/" + table_name))).toMap

def writeTmpDf(dfSeq: Seq[(DataFrame, String)]): Unit = 
    dfSeq.foreach{case (df: DataFrame, name: String) => df.write.mode("overwrite").parquet("../../resources/data/tmp/parquet/" + name)}

def schema_to_ddl(df: DataFrame): String = df.schema.toDDL.replace(" NOT NULL", "")

import org.apache.spark.sql.{functions=>f}
import org.apache.spark.sql.{types=>t}
difference: (l1: Seq[String], l2: Seq[String])Seq[org.apache.spark.sql.Column]
readTmpDf: (dfSeq: Seq[String])Map[String,org.apache.spark.sql.DataFrame]
writeTmpDf: (dfSeq: Seq[(org.apache.spark.sql.DataFrame, String)])Unit
schema_to_ddl: (df: org.apache.spark.sql.DataFrame)String


In [3]:
// NO MODIFICAR CONTENIDO DE ESTA CELDA

// Creación de sesión de Spark
val spark = SparkSession.builder
    .master("local[*]")
    .appName("ejercicio_9")
    .getOrCreate()

spark.conf.set("spark.sql.session.timeZone", "GMT-6")

// Carga de tablas requeridas
val RootPath = "../../resources/data/tmp/parquet/"
val namesList = Seq("06/movies", "06/ratings", "06/tags")
val dfMap = readTmpDf(namesList)

val moviesDf = dfMap("06/movies")
val ratingsDf = dfMap("06/ratings")
val tagsDf = dfMap("06/tags")

moviesDf.show(1, false)
ratingsDf.show(1)
tagsDf.show(1)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



tagsDf = [user_id: string, movie_id: string ... 2 more fields]


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@22df8770
RootPath: String = ../../resources/data/tmp/parquet/
namesList: Seq[String] = List(06/movies, 06/ratings, 06/tags)
dfMap: Map[String,org.apache.spark.sql.DataFrame] = Map(06/movies -> [movie_id: string, title: string ... 2 more fields], 06/ratings -> [user_id: string, movie_id: string ... 2 more fields], 06/tags -> [user_id: string, movie_id: string ... 2 more fields])
moviesDf: org.apache.spark.sql.DataFrame = [movie_id: string, title: string ... 2 more fields]
ratingsDf: org.apache.spark.sql.DataFrame = [user_id: string, movie_id: string ... 2 more fields]


[user_id: string, movie_id: string ... 2 more fields]

#### Actividad 1:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 2 de la sesión 07 - método calculate_rating_values" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### def calculateRatingValuesW(df: DataFrame): DataFrame

In [8]:
// TU CODIGO VA EN ESTA CELDA:

import org.apache.spark.sql.expressions.{Window, WindowSpec}

val windowCRRV: WindowSpec = Window.partitionBy(f.col("movie_id"))

def calculateRatingValuesW(df: DataFrame): DataFrame = {
    val crv_df = df.withColumn("avg_rating", f.round(f.avg("rating").over(windowCRRV), 2))
            .withColumn("stddev_rating", f.round(f.stddev_pop("rating").over(windowCRRV), 2))
            .withColumn("count_rating", f.count("rating").over(windowCRRV))
            .withColumn("min_time_rating", f.min("time").over(windowCRRV))
            .select("movie_id", "avg_rating", "stddev_rating", "count_rating", "min_time_rating")
            .distinct() //Returns a new DataFrame containing the distinct rows in this DataFrame
    return crv_df
}
    // ratingsDf modificar codigo interno

windowCRRV: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7af82a62
calculateRatingValuesW: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [10]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculateRatingValuesW(ratingsDf).show(2)
/*"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""*/

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows



In [11]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
import java.sql.Timestamp

val ratingValuesDf = calculateRatingValuesW(ratingsDf)

assert(ratingValuesDf.isInstanceOf[DataFrame])
assert(ratingValuesDf.columns.size == 5)
assert(ratingValuesDf.count() == 83239)

val data = ratingValuesDf
    .select(f.col("movie_id"),
            f.col("avg_rating").cast(t.DoubleType), 
            f.col("stddev_rating").cast(t.DoubleType),
            f.col("count_rating").cast(t.LongType),
            f.col("min_time_rating").cast(t.TimestampType))
    .filter(f.col("movie_id") === "296")
    .collect()(0)

assert(data.getAs[String]("movie_id") == "296")
assert(data.getAs[Double]("avg_rating") == 4.19)
assert(data.getAs[Double]("stddev_rating") == 0.95)
assert(data.getAs[Long]("count_rating") == 108756)
assert(data.getAs[Timestamp]("min_time_rating") == Timestamp.valueOf("1996-02-29 10:48:44.0"))

data = [296,4.19,0.95,108756,1996-02-29 10:48:44.0]


ratingValuesDf: org.apache.spark.sql.DataFrame = [movie_id: string, avg_rating: double ... 3 more fields]


[296,4.19,0.95,108756,1996-02-29 10:48:44.0]

In [12]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val dfs = Seq((ratingValuesDf, "09/ratings"))

writeTmpDf(dfs)

dfs = List(([movie_id: string, avg_rating: double ... 3 more fields],09/ratings))


List(([movie_id: string, avg_rating: double ... 3 more fields],09/ratings))

#### Actividad 2:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 3 de la sesión 07 - métodos getAct3Df1 y getAct3Df2" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### Primer tabla (utilizando función concat_ws o concat): def getAct3Df1W(df: DataFrame) -> DataFrame:
- ##### Segunda tabla (utilizando función struct): def getAct3Df2W(df: DataFrame) -> DataFrame:

In [14]:
//Declaración de windows:
val windowGA31: WindowSpec = Window.partitionBy(f.col("movie_id"), f.col("tag")) //aquí antes tenía un f.upper(f.col("tag")) pero no transformaba
val windowGA32: WindowSpec = Window.partitionBy(f.col("movie_id"))

windowGA32 = org.apache.spark.sql.expressions.WindowSpec@1b627ea7


windowGA31: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@73aa1495


org.apache.spark.sql.expressions.WindowSpec@1b627ea7

In [16]:
// TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES

def getAct3Df1W(df: DataFrame): DataFrame = {
    val dfMTTw = df
        .withColumn("tag_count", f.concat_ws(" : ", f.upper(f.col("tag")), f.count("tag").over(windowGA31)))
        .withColumn("min_tag", f.min("time").over(windowGA31))
        .select("movie_id", "tag_count", "min_tag")
        .distinct()
    val tabla1DFFw = dfMTTw
        .withColumn("tag_count", f.sort_array(f.collect_set("tag_count").over(windowGA32)))
        .withColumn("min_time_tag", f.min("min_tag").over(windowGA32))
        .select("movie_id", "tag_count", "min_time_tag")
        .distinct()
    return tabla1DFFw
}
    // ... transformaciones a tagsDf

def getAct3Df2W(df: DataFrame): DataFrame = {
    val dfMTTw2 = df
        .withColumn("tag", f.upper(f.col("tag")))
        .withColumn("count", f.count("tag").over(windowGA31))
        .withColumn("struct", f.struct(f.col("tag"), f.col("count")))
        .withColumn("min_tag", f.min("time").over(windowGA31))
        .select("movie_id", "struct", "min_tag")
        .distinct()
    val tabla2DFFw = dfMTTw2
        .withColumn("tag_count", f.sort_array(f.collect_set("struct").over(windowGA32)))
        .withColumn("min_time_tag", f.min("min_tag").over(windowGA32))
        .select("movie_id", "tag_count", "min_time_tag")
        .distinct()
    return tabla2DFFw
}
    // ... transformaciones a tagsDf

getAct3Df1W: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getAct3Df2W: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [17]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val act3Df1 = getAct3Df1W(tagsDf)
val act3Df2 = getAct3Df2W(tagsDf)

act3Df1.show(2, false)
act3Df2.show(2, false)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |2018-05-26 16:40:54|
|100070  |[{COMEDIAN, 2}, {COMEDY, 1}, {GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |

act3Df1: org.apache.spark.sql.DataFrame = [movie_id: string, tag_count: array<string> ... 1 more field]
act3Df2: org.apache.spark.sql.DataFrame = [movie_id: string, tag_count: array<struct<tag:string,count:bigint>> ... 1 more field]
"
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------...


In [19]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
import java.sql.Timestamp

val expectedValueDf1 = Seq(Row("100070",
                               List("COMEDIAN : 2",
                                    "COMEDY : 1",
                                    "GOOD HUMOUR : 1",
                                    "STRUGGLING CAREER : 1"),
                                   ("2017-05-19 17:17:36.0")))
val schemaDf1 = t.StructType(Seq(
    t.StructField("movie_id", t.StringType),
    t.StructField("tag_count", t.ArrayType(t.StringType)),
    t.StructField("min_time_tag", t.StringType)
))
assert(act3Df1.columns.size == 3)
assert(act3Df1.columns.toSeq.contains("movie_id"))
assert(act3Df1.columns.toSeq.contains("tag_count"))
assert(act3Df1.columns.toSeq.contains("min_time_tag"))
assert(schema_to_ddl(act3Df1.select("movie_id", "tag_count", "min_time_tag")) == "movie_id STRING,tag_count ARRAY<STRING>,min_time_tag TIMESTAMP")
assert(act3Df1.count() == 53452)
val testDf1 = spark.createDataFrame(spark.sparkContext.parallelize(expectedValueDf1), schemaDf1)
    .withColumn("min_time_tag", f.col("min_time_tag").cast(t.TimestampType))
assert(act3Df1.select("movie_id", "tag_count", "min_time_tag").filter(f.col("movie_id") === "100070").except(testDf1).count() == 0)

val expectedValueDf2 = Seq(Row("100070",
                               List(Row("COMEDIAN",2L),
                                    Row("COMEDY",1L),
                                    Row("GOOD HUMOUR",1L),
                                    Row("STRUGGLING CAREER",1L)),
                                   ("2017-05-19 17:17:36.0")))
val schemaDf2 = t.StructType(Seq(
    t.StructField("movie_id", t.StringType),
    t.StructField("tag_count", t.ArrayType(t.StructType(Seq(
        t.StructField("tag", t.StringType),
        t.StructField("count", t.LongType)
    )))),
    t.StructField("min_time_tag", t.StringType)
))
assert(act3Df2.columns.size == 3)
assert(act3Df2.columns.toSeq.contains("movie_id"))
assert(act3Df2.columns.toSeq.contains("tag_count"))
assert(act3Df2.columns.toSeq.contains("min_time_tag"))
assert(schema_to_ddl(act3Df2.select("movie_id", "tag_count", "min_time_tag")) == "movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,min_time_tag TIMESTAMP")
assert(act3Df2.count() == 53452)
val testDf2 = spark.createDataFrame(spark.sparkContext.parallelize(expectedValueDf2), schemaDf2)
    .withColumn("min_time_tag", f.col("min_time_tag").cast(t.TimestampType))
assert(act3Df2.select("movie_id", "tag_count", "min_time_tag").filter(f.col("movie_id") === "100070").except(testDf2).count() == 0)

schemaDf2 = StructType(StructField(movie_id,StringType,true),StructField(tag_count,ArrayType(StructType(Struc...


lastException: Throwable = null
expectedValueDf1: Seq[org.apache.spark.sql.Row] = List([100070,List(COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1),2017-05-19 17:17:36.0])
schemaDf1: org.apache.spark.sql.types.StructType = StructType(StructField(movie_id,StringType,true),StructField(tag_count,ArrayType(StringType,true),true),StructField(min_time_tag,StringType,true))
testDf1: org.apache.spark.sql.DataFrame = [movie_id: string, tag_count: array<string> ... 1 more field]
expectedValueDf2: Seq[org.apache.spark.sql.Row] = List([100070,List([COMEDIAN,2], [COMEDY,1], [GOOD HUMOUR,1], [STRUGGLING CAREER,1]),2017-05-19 17:17:36.0])


StructType(StructField(movie_id,StringType,true),StructField(tag_count,ArrayType(StructType(Struc...

In [20]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val dfs = Seq((act3Df1, "09/tags_p1"),
              (act3Df2, "09/tags_p2"))

writeTmpDf(dfs)

dfs = List(([movie_id: string, tag_count: array<string> ... 1 more field],09/tags_p1), ([movie_id: string, tag_count: array<struct<tag:string,count:bigint>> ... 1 more field],09/tags_p2))


List(([movie_id: string, tag_count: array<string> ... 1 more field],09/tags_p1), ([movie_id: string, tag_count: array<struct<tag:string,count:bigint>> ... 1 more field],09/tags_p2))

In [21]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val lastMoviesDf = moviesDf
    .join(act3Df2, Seq("movie_id"), "left")
    .join(ratingValuesDf, Seq("movie_id"), "left")

lastMoviesDf = [movie_id: string, title: string ... 8 more fields]


[movie_id: string, title: string ... 8 more fields]

In [22]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val dfs = Seq((lastMoviesDf, "09/movies"))

writeTmpDf(dfs)

dfs = List(([movie_id: string, title: string ... 8 more fields],09/movies))


List(([movie_id: string, title: string ... 8 more fields],09/movies))

#### Actividad 3:
##### TO DO ->    La última actividad a realizar es la siguiente:
- ##### El cliente nos ha solicitado generar una tabla donde se muestre el top de peliculas ranqueadas por genero, para realizar esto se realizan los siguientes pasos:
    - ##### Las peliculas que entran a la parte del ranqueo deben cumplir con las siguientes condiciones:
        -  count_rating > 1000
        -  avg_rating >= 4.2
        -  stddev_rating < 2
    - #####  Descomponer la columna "genres" en una columna llamada "genre"
    - ##### Agregar la columna "top" donde se asignará el valor de la función "rank" de Spark tomando las siguientes caracteristicas:
        - particionar por: "genre"
        - order por: avg_rating DESC, stddev_rating ASC, count_rating DESC
##### El esquema resultante deberá ser:
* |-- title: string
* |-- year: integer
* |-- genre: string
* |-- top: integer

In [24]:
// TU CODIGO VA EN ESTA CELDA:

//val topMoviesDf = lastMoviesDf // Transformaciones a lastMoviesDf

val dfAct3 = lastMoviesDf
    .filter(
        (f.col("count_rating") > 1000) &&
        (f.col("avg_rating") >= 4.2) &&
        (f.col("stddev_rating") < 2)
    )
    .select(
        f.col("title"),
        f.col("year"),
        f.col("avg_rating"),
        f.col("stddev_rating"),
        f.col("count_rating"),
        f.explode(f.col("genres")).alias("genre")
    )

// TEST
// df_act3.show(5)

val windowAct3: WindowSpec = Window
    .partitionBy(f.col("genre"))
    .orderBy(
        f.col("avg_rating").desc,
        f.col("stddev_rating").asc,
        f.col("count_rating").desc
    )

val topMoviesDf = dfAct3
    .withColumn("top", f.rank().over(windowAct3))
    .select("title", "year", "genre", "top")

topMoviesDf = [title: string, year: int ... 2 more fields]


dfAct3: org.apache.spark.sql.DataFrame = [title: string, year: int ... 4 more fields]
windowAct3: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@55d311c9


[title: string, year: int ... 2 more fields]

In [25]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA

topMoviesDf.show(5)
"""
Ejemplo de salida esperada (hay que mantener mismo orden de columnas):
+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows
"""

+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows



"
Ejemplo de salida esperada (hay que mantener mismo orden de columnas):
+--------------------+----+---------+---+
               title|year|    genre|top|
+--------------------+----+---------+---+
Band of Brothers ...|2001|   Action|  1|
Seven Samurai (Sh...|1954|   Action|  2|
   Fight Club (1999)|1999|   Action|  3|
Over the Garden W...|2013|Adventure|  1|
Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows
"


In [26]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA

val expectedTopMovies = Seq(
    Row("Band of Brothers (2001)", 2001, "Action", 1),
    Row("Seven Samurai (Shichinin no samurai) (1954)", 1954, "Action", 2),
    Row("Fight Club (1999)", 1999, "Action", 3),
    Row("Over the Garden Wall (2013)", 2013, "Adventure", 1),
    Row("Seven Samurai (Shichinin no samurai) (1954)", 1954, "Adventure", 2),
    Row("Spirited Away (Sen to Chihiro no kamikakushi) (2001)", 2001, "Adventure", 3),
    Row("Over the Garden Wall (2013)", 2013, "Animation", 1),
    Row("Spirited Away (Sen to Chihiro no kamikakushi) (2001)", 2001, "Animation", 2),
    Row("Parasite (2019)", 2019, "Comedy", 1),
    Row("Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)", 1964, "Comedy", 2),
    Row("Shawshank Redemption, The (1994)", 1994, "Crime", 1),
    Row("Godfather, The (1972)", 1972, "Crime", 2),
    Row("Usual Suspects, The (1995)", 1995, "Crime", 3),
    Row("Godfather: Part II, The (1974)", 1974, "Crime", 4),
    Row("Fight Club (1999)", 1999, "Crime", 5),
    Row("Planet Earth (2006)", 2006, "Documentary", 1),
    Row("Planet Earth II (2016)", 2016, "Documentary", 2),
    Row("Blue Planet II (2017)", 2017, "Documentary", 3),
    Row("The Blue Planet (2001)", 2001, "Documentary", 4),
    Row("Shawshank Redemption, The (1994)", 1994, "Drama", 1),
    Row("Band of Brothers (2001)", 2001, "Drama", 2),
    Row("Parasite (2019)", 2019, "Drama", 3),
    Row("Godfather, The (1972)", 1972, "Drama", 4),
    Row("Twin Peaks (1989)", 1989, "Drama", 5),
    Row("12 Angry Men (1957)", 1957, "Drama", 6),
    Row("Godfather: Part II, The (1974)", 1974, "Drama", 7),
    Row("Over the Garden Wall (2013)", 2013, "Drama", 8),
    Row("Seven Samurai (Shichinin no samurai) (1954)", 1954, "Drama", 9),
    Row("Fight Club (1999)", 1999, "Drama", 10),
    Row("Schindler's List (1993)", 1993, "Drama", 11),
    Row("One Flew Over the Cuckoo's Nest (1975)", 1975, "Drama", 12),
    Row("Lives of Others, The (Das leben der Anderen) (2006)", 2006, "Drama", 13),
    Row("Casablanca (1942)", 1942, "Drama", 14),
    Row("Spirited Away (Sen to Chihiro no kamikakushi) (2001)", 2001, "Fantasy", 1),
    Row("Twin Peaks (1989)", 1989, "Mystery", 1),
    Row("Usual Suspects, The (1995)", 1995, "Mystery", 2),
    Row("Rear Window (1954)", 1954, "Mystery", 3),
    Row("Lives of Others, The (Das leben der Anderen) (2006)", 2006, "Romance", 1),
    Row("Casablanca (1942)", 1942, "Romance", 2),
    Row("Usual Suspects, The (1995)", 1995, "Thriller", 1),
    Row("Fight Club (1999)", 1999, "Thriller", 2),
    Row("Rear Window (1954)", 1954, "Thriller", 3),
    Row("Lives of Others, The (Das leben der Anderen) (2006)", 2006, "Thriller", 4),
    Row("Band of Brothers (2001)", 2001, "War", 1),
    Row("Schindler's List (1993)", 1993, "War", 2),
    Row("Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)", 1964, "War", 3)
)

topMoviesDf
    .select(f.col("title"),f.col("year").cast(t.IntegerType),f.col("genre"),f.col("top").cast(t.IntegerType))
    .orderBy(f.col("genre").asc, f.col("top").asc)
    .collect()
    .zip(expectedTopMovies)
    .foreach(tuple => {
        assert(tuple._1.getAs[String]("title") == tuple._2.getAs[String](0))
        assert(tuple._1.getAs[Int]("year") == tuple._2.getAs[Int](1))
        assert(tuple._1.getAs[String]("genre") == tuple._2.getAs[String](2))
        assert(tuple._1.getAs[Int]("top") == tuple._2.getAs[Int](3))
    })

expectedTopMovies = List([Band of Brothers (2001),2001,Action,1], [Seven Samurai (Shichinin no samurai) (1954),1954,Action,2], [Fight Club (1999),1999,Action,3], [Over the Garden Wall (2013),2013,Adventure,1], [Seven Samurai (Shichinin no samurai) (1954),1954,Adventure,2], [Spirited Away (Sen to Chihiro no kamikakushi) (2001),2001,Adventure,3], [Over the Garden Wall (2013),2013,Animation,1], [Spirited Away (Sen to Chihiro no kamikakushi) (2001),2001,Animation,2], [Parasite (2019),2019,Comedy,1], [Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),1964,Comedy,2], [Shawshank Redemption, The (1994),1994,Crime,1], [Godfather, The (1972),1972,Crime,2], [Usual Suspects, The (1995),1995,Crime,3], [Godfather: Part II, The (1974),1974,Crime...


List([Band of Brothers (2001),2001,Action,1], [Seven Samurai (Shichinin no samurai) (1954),1954,Action,2], [Fight Club (1999),1999,Action,3], [Over the Garden Wall (2013),2013,Adventure,1], [Seven Samurai (Shichinin no samurai) (1954),1954,Adventure,2], [Spirited Away (Sen to Chihiro no kamikakushi) (2001),2001,Adventure,3], [Over the Garden Wall (2013),2013,Animation,1], [Spirited Away (Sen to Chihiro no kamikakushi) (2001),2001,Animation,2], [Parasite (2019),2019,Comedy,1], [Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),1964,Comedy,2], [Shawshank Redemption, The (1994),1994,Crime,1], [Godfather, The (1972),1972,Crime,2], [Usual Suspects, The (1995),1995,Crime,3], [Godfather: Part II, The (1974),1974,Crime...

In [27]:
// NO MODIFICAR EL CONTENIDO DE ESTA CELDA
val dfs = Seq((topMoviesDf, "09/top_movies"))

writeTmpDf(dfs)

dfs = List(([title: string, year: int ... 2 more fields],09/top_movies))


List(([title: string, year: int ... 2 more fields],09/top_movies))